<a href="https://colab.research.google.com/github/CodeMonkey01/DataMiningI/blob/main/ANN/Option_A/ANN-FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd '/content/drive/MyDrive/'

    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/MyDrive
Not connected to a GPU


In [2]:
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 10.7 MB/s 
     |████████████████████████████████| 511.7 MB 7.4 kB/s 
     |████████████████████████████████| 5.8 MB 62.3 MB/s 
     |████████████████████████████████| 438 kB 55.7 MB/s 
     |████████████████████████████████| 1.6 MB 57.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset.txt')
df.describe()

,text,humor
count,200000,200000
unique,200000,2
top,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
freq,1,100000


In [5]:
# Transform class from Boolean to integer value
df['class']=df['humor'].apply(lambda x: 1 if x==True else 0)

NameError: ignored

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

MAX_LEN = 128
#pad_to_max_length=True,
df['bert_preprocessed']=df['text'].apply(lambda x: " ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(x, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))))

In [ ]:
df.head()

In [ ]:
X = df['bert_preprocessed']
Y = df['class']

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4")

In [ ]:
def build_model() -> tf.keras.Model:
    # Bert layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[text_input], outputs = [l])

    #model.summary()

    return model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

ADAM_LEARNING_RATE = 0.0044
ADAM_EPSILON = 1e-05

EPOCH = 50

BATCH_SIZE = 64

PATIENCE = 2

base_path = "/content/drive/MyDrive/Data Mining/bert/"

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

def train_evaluate(model, run, x_train, y_train, x_test, y_test):
    STAMP = 'model_run_%.2f'%(run)
    early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE)
    bst_model_path = base_path + STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=ADAM_LEARNING_RATE, epsilon=ADAM_EPSILON),
                  loss=tf.keras.losses.BinaryCrossentropy(),                  
                  metrics=METRICS)
    

    X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, test_size=0.2)

    hist = model.fit(X_tra, y_tra, validation_data=(X_val, y_val), epochs=EPOCH, batch_size=BATCH_SIZE, shuffle=True, callbacks=[early_stopping, model_checkpoint])

    return model.evaluate(x_test, y_test)

kFold = StratifiedKFold(n_splits=4)
accuracy_list = []
precision_list = []
recall_list = []
i = 1
for train, test in kFold.split(X, Y):
    model = build_model()
    _, accuracy, precision, recall = train_evaluate(model, i, X[train], Y[train], X[test], Y[test])
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    i = i + 1

print(f"Accuracy: {sum(accuracy_list) / len(accuracy_list)}")
print(f"Precision: {sum(precision_list) / len(precision_list)}")
print(f"Recall: {sum(recall_list) / len(recall_list)}")